In [41]:
from ydata_profiling import ProfileReport
import pandas as pd

In [ ]:
df = pd.read_parquet(r"../data/types_fixed_german_credit_data.parquet")
df.head()

,Duration,Credit amount,Purpose,Job,Sex,Saving accounts,Housing,Risk,Age
0,24,3632,car,2.0,female,little,rent,good,22
1,24,2603,car,2.0,female,rich,rent,good,28
2,12,902,education,2.0,male,little,rent,bad,21
3,9,654,car,1.0,male,little,own,bad,28
4,21,3414,education,2.0,male,little,own,bad,26


: 

In [ ]:
report = ProfileReport(df.sample(frac=0.9), explorative=True)
report.to_file(r"../data/eda.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]